### Setup

##### !! NOTE !!
To run this code you need the data from [chicagobooth](https://www.chicagobooth.edu/research/kilts/datasets/dominicks), specifically the data for shampoo.

The filles needed is:

**Customoer Count File** - (ccount(stata).zip)

**Store-Level Demographics File** - (demo(stata).zip)

And from the **category file** you need to find **shampoo**, and download **UPC.csv File** and **Movement.csv File**

And all of this needs to be saved in a folder called **"data"** in the same folder you are running this script from, running the **"create_folders" function** will automatically create the folder

In [2]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import requests
import os
import chardet
import time
import csv
import gc

from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as mticker
import matplotlib.pylab as pylab

import sympy as sp
from sympy.solvers import solve

from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

from cycler import cycler

# custome plot style
params  = {
"lines.linewidth": 1.5,

"legend.fancybox": "true",

"axes.prop_cycle": cycler('color', ["#ffa822","#1ac0c6","#ff6150","#30B66A","#B06AFF","#FF21E1"]),
"axes.facecolor": "#2b2b2b",
"axes.axisbelow": "true",
"axes.grid": "true",
"axes.edgecolor": "#2b2b2b",
"axes.linewidth": 0.5,
"axes.labelpad": 0,

"patch.edgecolor": "#2b2b2b",
"patch.linewidth": 0.5,

"grid.linestyle": "--",
"grid.linewidth": 0.5,
"grid.color": "#b8aba7",

"xtick.major.size": 0,
"xtick.minor.size": 0,
"ytick.major.size": 0,
"ytick.minor.size": 0,

"font.family":"monospace",
"font.size":10.0,
"text.color": "#FFE9E3",
"axes.labelcolor": "#b8aba7",
"xtick.color": "#b8aba7",
"ytick.color": "#b8aba7",

"savefig.edgecolor": "#2b2b2b",
"savefig.facecolor": "#2b2b2b",

"figure.subplot.left": 0.08,
"figure.subplot.right": 0.95,
"figure.subplot.bottom": 0.09,
"figure.facecolor": "#2b2b2b"}

pylab.rcParams.update(params)
print("finish")

finish


In [7]:
# Collect files from folder, if file type equals file_type
def get_files(folder, file_type):
    file_paths = []
    for file in os.listdir(folder):
        if file.endswith(f"{file_type}"):
            file_paths.append([os.path.join(folder, file), file_type])
    return file_paths

def get_encoder(file_path, chunksize = 10_000):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read(chunksize))
    return result['encoding']

# NOTE TO SELF, STRAGE ERROR HERE, MIGHT BE A DUPLICATE ERROR
# Load, filter in chunks and Convert to csv
def load_and_filter_file(input_file, temp_path, filter_func:list, file_type=".csv", chunksize=10_000, new_file_name = ""):
    temp_file = os.path.join(temp_path,new_file_name)
    if new_file_name == "":
        # Extract the file name from the input_file path
        input_file_name = os.path.basename(input_file)

        # Create a temp_file path by combining temp_path and input_file_name
        file_name_without_ext, file_ext = os.path.splitext(input_file_name)
        temp_file = os.path.join(temp_path, f"{file_name_without_ext}_temp.csv")
        
    # Had to fix the decoding because 'invalid continuation byte' that utf-8 can't decode. And manual attempt to fix it did not reveal byte 0xd5
    encodings = ["utf-8", "ISO-8859-1", "cp1252", "latin1"]
    success = False

    for encoding in encodings:
        try:
            if file_type.lower() == '.csv':
                reader = pd.read_csv(input_file, chunksize=chunksize, encoding=encoding)
            elif file_type.lower() == '.dta':
                reader = pd.read_stata(input_file, chunksize=chunksize)
            else:
                raise ValueError("Unsupported file type. Supported types are 'csv' and 'dta'.")

            for i, chunk in enumerate(reader):
                filtered_chunk = chunk
                for func in filter_func:
                    filtered_chunk = func(filtered_chunk)
                if i == 0:
                    filtered_chunk.to_csv(temp_file, index=False, mode='w')
                else:
                    filtered_chunk.to_csv(temp_file, index=False, mode='a', header=False)

            success = True
            print(f"Succes with the encoding '{encoding}', file {temp_file} now created")
            break

        except UnicodeDecodeError:
            print(f"Failed to read the file with encoding '{encoding}', trying the next one...")

    if not success:
        raise ValueError("None of the attempted encodings were successful in reading the file.")
            
# Merge csv files
"""
def merge_csv_files(file1, file2, output_file, merge_on=None, merge_dtype=None, chunksize=10000):
    if (merge_on is not None) and (type(merge_on) != list):  # Fix the condition here
        merge_on = [merge_on]
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f_out:
        writer = None
        for chunk1 in pd.read_csv(file1, chunksize=chunksize):
            for chunk2 in pd.read_csv(file2, chunksize=chunksize):
                if merge_dtype is not None:
                    for column in merge_on:
                        chunk1[column] = chunk1[column].astype(merge_dtype)
                        chunk2[column] = chunk2[column].astype(merge_dtype)

                merged_chunk = pd.merge(chunk1, chunk2, on=merge_on) if merge_on else pd.concat([chunk1, chunk2], axis=1)

                if writer is None:
                    writer = csv.DictWriter(f_out, fieldnames=merged_chunk.columns)
                    writer.writeheader()

                for row in merged_chunk.to_dict(orient='records'):
                    writer.writerow(row)
"""

def merge_csv_files(file1, file2, output_file, merge_on= None, merge_dtype=None, chunksize =10000):
    if (merge_on is not None) and (type(merge_on) != list):  # Fix the condition here
        merge_on = [merge_on]
    
    with open(output_file, 'w', newline='', encoding='utf-8') as f_out:
        writer = None
        
        # Read the entire file2 into memory
        file2_data = pd.read_csv(file2)
        
        if merge_dtype is not None:
            for column in merge_on:
                file2_data[column] = file2_data[column].astype(merge_dtype)
        
        for chunk1 in pd.read_csv(file1, chunksize=chunksize):
            if merge_dtype is not None:
                for column in merge_on:
                    chunk1[column] = chunk1[column].astype(merge_dtype)

            merged_chunk = pd.merge(chunk1, file2_data, on=merge_on) if merge_on else pd.concat([chunk1, file2_data], axis=1)

            if writer is None:
                writer = csv.DictWriter(f_out, fieldnames=merged_chunk.columns)
                writer.writeheader()

            for row in merged_chunk.to_dict(orient='records'):
                writer.writerow(row)
                

                    
# Run time test function
def time_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"{func.__name__} took {elapsed_time:.2f} seconds to run.")
    return result

# Folder check and creation 
def create_folders(folder_paths):
    for folder_path in folder_paths:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder created: {folder_path}")
        else:
            print(f"Folder already exists: {folder_path}")

In [4]:
# List of filters
def empty_filter_func(chunk):
    # Empty filter for test
    return chunk

def filter_empty_to_0_and_dropna(chunk):
    # Replace empty values with 0 if the column is numeric, otherwise with NaN
    for col in chunk.columns:
        if pd.api.types.is_numeric_dtype(chunk[col]):
            chunk[col] = chunk[col].replace('', 0).fillna(0)
        else:
            chunk[col] = chunk[col].replace('', np.nan)
    
    # Drop rows with NaN values
    filtered_chunk = chunk.dropna()
    
    return filtered_chunk


def filter_remove_empty_and_nan(chunk):
    filtered_chunk = chunk.replace('', np.nan)
    # Remove rows with NaN values from the DataFrame
    filtered_chunk = filtered_chunk.dropna()
    return filtered_chunk

def filter_header_up(chunk):
    # Turn header to upper
    chunk.columns = map(str.upper, chunk.columns)
    return chunk

def filter_week(chunk):
    # The data begins from week 128 (02/20/92). To filter for the year 1993, we select week 173 to 225.
    start_week = 173
    end_week = 225

    # Create a copy of the chunk to avoid the warning
    chunk_copy = chunk.copy()

    # Modify the 'WEEK' column in the copied chunk
    chunk_copy['WEEK'] = chunk_copy['WEEK'].astype(int)

    filtered_chunk = chunk_copy[(chunk_copy['WEEK'] >= start_week) & (chunk_copy['WEEK'] <= end_week)]
    return filtered_chunk

def filter_move_above_one(chunk):
    filtered_chunk = chunk[(chunk['MOVE'] > 0)]
    return filtered_chunk

def filter_out_bad_data(chunk):
    filtered_chunk = chunk[(chunk['OK'] > 0)]
    return filtered_chunk

def filter_keep_columns(chunk, columns_to_keep = ["STORE", "COSMETIC", "HABA","PHARMACY","WEEK"]):
    # Keep only the specified columns in the DataFrame
    filtered_chunk = chunk[columns_to_keep]
    return filtered_chunk

def remove_column(chunk):
    column_names = ["GINI","LIFT5","RATIO5"]
    for column_name in column_names:
        if column_name in chunk.columns:
            chunk = chunk.drop(column_name, axis=1)
    return chunk

def combine_same_week_and_store(chunk):
    # Combine rows with the same value in the "WEEK" column
    combined_chunk = chunk.groupby(["WEEK","STORE"]).sum().reset_index()
    return combined_chunk


### Filter and Clean Data

In [5]:
folder_path = "data/" # folder with data
folder_path_temp = "data_temp/" # folder with temp data
folder_path_clean = "data_clean/" # folder with clean
chunk_size = 10_000 # chunks of data loaderd in memory

create_folders([folder_path_temp,folder_path_clean])

file_paths = get_files(folder_path,".csv")
file_paths.extend(get_files(folder_path,".dta"))

print(file_paths)

Folder already exists: data_temp/
Folder already exists: data_clean/
[['data/upcsha.csv', '.csv'], ['data/wsha.csv', '.csv'], ['data/ccount.dta', '.dta'], ['data/demo.dta', '.dta']]


#### upc data

In [8]:
filter_func_list = [filter_header_up,filter_remove_empty_and_nan]
time_function(load_and_filter_file, file_paths[0][0], folder_path_temp, filter_func_list, file_type=file_paths[0][1], chunksize = chunk_size)

Failed to read the file with encoding 'utf-8', trying the next one...
Succes with the encoding 'ISO-8859-1', file data_temp/upcsha_temp.csv now created
load_and_filter_file took 0.02 seconds to run.


#### walk data

In [9]:
filter_func_list = [filter_header_up,filter_remove_empty_and_nan,filter_week,filter_move_above_one,filter_out_bad_data]
time_function(load_and_filter_file, file_paths[1][0], folder_path_temp, filter_func_list, file_type=file_paths[1][1], chunksize = chunk_size)

Succes with the encoding 'utf-8', file data_temp/wsha_temp.csv now created
load_and_filter_file took 31.21 seconds to run.


#### custumer count

In [10]:
filter_func_list = [filter_header_up,filter_remove_empty_and_nan,filter_week,filter_keep_columns, combine_same_week_and_store]
time_function(load_and_filter_file, file_paths[2][0], folder_path_temp, filter_func_list, file_type=file_paths[2][1], chunksize = chunk_size)

Succes with the encoding 'utf-8', file data_temp/ccount_temp.csv now created
load_and_filter_file took 1.02 seconds to run.


#### demo

In [11]:
filter_func_list = [filter_header_up, remove_column,filter_empty_to_0_and_dropna]
time_function(load_and_filter_file, file_paths[3][0], folder_path_temp, filter_func_list, file_type=file_paths[3][1], chunksize = chunk_size)

Succes with the encoding 'utf-8', file data_temp/demo_temp.csv now created
load_and_filter_file took 0.34 seconds to run.


### Merge Data

In [12]:
merge_file_main = "data_temp/wsha_temp.csv"
merge_file_sec = "data_temp/upcsha_temp.csv"
merge_file_out = "data_clean/wsha_upcsha.csv"
time_function(merge_csv_files,merge_file_main, merge_file_sec, merge_file_out, merge_on="UPC", merge_dtype=int, chunksize=chunk_size)

merge_csv_files took 3.50 seconds to run.


In [13]:
merge_file_main = "data_clean/wsha_upcsha.csv"
merge_file_sec = "data_temp/ccount_temp.csv"
merge_file_out = "data_clean/wsha_upcsha_ccount.csv"
time_function(merge_csv_files,merge_file_main, merge_file_sec, merge_file_out, merge_on=["STORE","WEEK"], merge_dtype=int, chunksize=chunk_size)

merge_csv_files took 4.32 seconds to run.


In [14]:
merge_file_main = "data_clean/wsha_upcsha_ccount.csv"
merge_file_sec = "data_temp/demo_temp.csv"
merge_file_out = "data_clean/wsha_upcsha_ccount_demo.csv"
time_function(merge_csv_files,merge_file_main, merge_file_sec, merge_file_out, merge_on="STORE", merge_dtype=int, chunksize=chunk_size)

merge_csv_files took 107.64 seconds to run.


### Select Relevant Column

In [18]:
def top_n_values_with_names(input_file, column_number, column_name, column_desc, n=5):
    df = pd.read_csv(input_file)

    # Sort the DataFrame based on the values in the specified column_number
    sorted_df = df.sort_values(by=column_number, ascending=False)

    # Get the top N rows from the sorted DataFrame
    top_n_rows = sorted_df.head(n)

    # Create a list with the top N values and their corresponding names from column_name
    result = top_n_rows[[column_number, column_name, column_desc]].values.tolist()

    return result

# Example usage
input_file = 'data_clean/wsha_upcsha.csv'
column_number = 'MOVE'
column_name = 'UPC'
column_desc = "DESCRIP"
n = 5
top_n_values = top_n_values_with_names(input_file, column_number, column_desc,column_name, n)
print(top_n_values)


[[96, 'WHITE RAIN SHMP RG B', 445092873], [80, '#WHT RAIN PLUS X/BOD', 445098233], [77, 'WHITE RAIN SHAMP DRY', 445092945], [69, '*PERT PLUS NORM', -594967207], [69, 'WHITE RAIN COND CLAR', 445092927]]


In [19]:
selected_columns = ["UPC","STORE", "WEEK","MOVE","PRICE","QTY","PROFIT","SALE","DESCRIP","CASE","COSMETIC","HABA","PHARMACY","INCOME","HSIZEAVG","HSIZE1","HSIZE2","HSIZE34","HHLARGE","SINGLE","RETIRED","UNEMP","WORKWOM","WRKCH5","WRKCH17","NWRKCH5","NWRKCH17","WRKCH","NWRKCH","WRKWNCH"]

def filter_select_columns(chunk, columns_to_keep = selected_columns):
    # Keep only the specified columns in the DataFrame
    filtered_chunk = chunk[columns_to_keep]
    return filtered_chunk

def filter_upc(chunk):
    filtered_chunk = chunk[(chunk['UPC'].isin([445092873,445098233,445092945,-594967207,445092927]))]
    return filtered_chunk

filter_func_list = [filter_select_columns,filter_upc]
file_main = "data_clean/wsha_upcsha_ccount_demo.csv"
file_out = "shampoo_sale_data.csv"
time_function(load_and_filter_file, file_main, folder_path_clean, filter_func_list, chunksize = chunk_size, new_file_name = file_out)

Succes with the encoding 'utf-8', file data_clean/shampoo_sale_data.csv now created
load_and_filter_file took 12.00 seconds to run.
